# Estradas e Trajetos

## Exercícios com SQL

Partindo do problema desenvolvido no notebook de modelagem (vide diretório `model/estradas`), um sistema de mapeamento de estradas e trajetos precisa gerenciar e relacionar os seguintes elementos: cidades, estradas e trajetos entre cidades. A seguir a descrição o papel de cada elemento:
* _Cidades_: mantém um cadastro de cidades.
* _Estradas_: registra estradas que ligam uma cidade (cidade_origem) a outra (cidade_destino), bem como sua quilometragem.
* _Trajeto_: cada registro da tabela Trajeto identifica um trajeto, que consiste em uma sequência ordenada de estradas que ligam duas cidades (cidade_origem e cidade_destino), por exemplo, um trajeto entre Salvador e Curitiba, pode envolver uma sequência de estradas: Salvador-Belo Horizonte, Belo Horizonte-São Paulo e São Paulo-Curitiba. Um Trajeto agrega um conjunto de Segmentos.
* _Segmento_: associa estradas a trajetos. O campo ordem é um campo numérico sequencial (iniciado de 1 para cada trajeto) usado para ordenar os segmentos (estradas) dentro de um trajeto.

No notebook de modelagem foram concebidos os seguintes modelos ER e relacional:

### Modelo ER

![ER](er-estradas.png)

### Esquema Relacional
As chaves primárias têm underscore antes e depois `_chave_`:
~~~
Cidade(_nome_)

Estrada(_nome_, km, origem, destino)
  CHE (origem) -> Cidade
  CHE (destino) -> Cidade

Trajeto(_id_, origem, destino)
  CHE (origem) -> Cidade
  CHE (destino) -> Cidade

Segmento(_idTrajeto_, _nomeEstrada_, ordem)
  CHE (idTrajeto) -> Trajeto
  CHE(nomeEstrada) -> Estrada
~~~

Considere que acrescentamos o atributo `id` além de `nome` para `Cidade` e `Estrada`, permitindo que Cidades e Estradas diferentes que tenham o mesmo nome possam se diferenciar. A partir disso, foi produzido o seguinte modelo UML e mapeamento para o modelo relacional representado graficamente:

### Modelo UML modificado

![UML](uml-estradas.png)

### Modelo Relacional modificado (representado graficamente)

![Relacional](relacional-estradas.png)

A seguir são apresentadas a criação das tabelas desse modelo em SQL.

In [1]:
%defaultDatasource jdbc:h2:file:~/data/estradas/estradas

DROP View IF EXISTS Maior;
DROP View IF EXISTS Distancia;

DROP Table IF EXISTS Cidade;
DROP Table IF EXISTS Estrada;
DROP Table IF EXISTS Trajeto;
DROP Table IF EXISTS Segmento;

In [2]:
CREATE TABLE Cidade (
  cidadeid VARCHAR(5) NOT NULL ,
  nome VARCHAR(100) ,
  PRIMARY KEY (cidadeid) );

INSERT INTO Cidade VALUES ('SSA', 'Salvador');
INSERT INTO Cidade VALUES ('SP',  'Sao Paulo');
INSERT INTO Cidade VALUES ('RJ',  'Rio de Janeiro');
INSERT INTO Cidade VALUES ('FLO', 'Florianopolis');
INSERT INTO Cidade VALUES ('CUR', 'Curitiba');

CREATE TABLE Estrada (
  estradaid VARCHAR(5) NOT NULL ,
  nome VARCHAR(100) ,
  cidade_origem VARCHAR(5) NOT NULL ,
  cidade_destino VARCHAR(5) NOT NULL ,
  extensao_km INT ,
  PRIMARY KEY (estradaid) ,
  FOREIGN KEY (cidade_origem )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  FOREIGN KEY (cidade_destino )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);

INSERT INTO Estrada VALUES ('4545', 'Estrada 4545', 'SSA', 'RJ', 1800);
INSERT INTO Estrada VALUES ('2835', 'Estrada 2835', 'RJ', 'SP', 400);
INSERT INTO Estrada VALUES ('1012', 'Estrada 1012', 'SP', 'FLO', 600);
INSERT INTO Estrada VALUES ('1212', 'Estrada 1212', 'SP', 'CUR', 800);
INSERT INTO Estrada VALUES ('6032', 'Estrada 6032', 'FLO', 'CUR', 200);

CREATE TABLE Trajeto (
  trajetoid VARCHAR(5) NOT NULL ,
  cidade_origem VARCHAR(5) NOT NULL ,
  cidade_destino VARCHAR(5) NOT NULL ,
  PRIMARY KEY (trajetoid) ,
  FOREIGN KEY (cidade_origem )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  FOREIGN KEY (cidade_destino )
    REFERENCES Cidade (cidadeid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);

INSERT INTO Trajeto VALUES ('TJ001', 'SSA', 'SP');
INSERT INTO Trajeto VALUES ('TJ002', 'SSA', 'FLO');
INSERT INTO Trajeto VALUES ('TJ003', 'SP', 'CUR');
INSERT INTO Trajeto VALUES ('TJ004', 'SP', 'CUR');
INSERT INTO Trajeto VALUES ('TJ005', 'RJ', 'CUR');

CREATE TABLE Segmento (
  trajetoid VARCHAR(5) NOT NULL ,
  estradaid VARCHAR(5) NOT NULL ,
  ordem INT ,
  PRIMARY KEY (trajetoid, estradaid) ,
  FOREIGN KEY (estradaid )
    REFERENCES Estrada (estradaid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  FOREIGN KEY (trajetoid )
    REFERENCES Trajeto (trajetoid )
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);

INSERT INTO Segmento VALUES ('TJ001', '4545', 1);
INSERT INTO Segmento VALUES ('TJ001', '2835', 2);
INSERT INTO Segmento VALUES ('TJ002', '4545', 1);
INSERT INTO Segmento VALUES ('TJ002', '2835', 2);
INSERT INTO Segmento VALUES ('TJ002', '1012', 3);
INSERT INTO Segmento VALUES ('TJ003', '4545', 1);
INSERT INTO Segmento VALUES ('TJ003', '2835', 2);
INSERT INTO Segmento VALUES ('TJ003', '1012', 3);
INSERT INTO Segmento VALUES ('TJ004', '4545', 1);
INSERT INTO Segmento VALUES ('TJ004', '2835', 2);
INSERT INTO Segmento VALUES ('TJ004', '1012', 3);
INSERT INTO Segmento VALUES ('TJ004', '6032', 4);
INSERT INTO Segmento VALUES ('TJ005', '1012', 1);
INSERT INTO Segmento VALUES ('TJ005', '6032', 2);

SELECT * FROM Cidade;
SELECT * FROM Estrada;
SELECT * FROM Trajeto;
SELECT * FROM Segmento;

# Exercício 1
Escreva uma sentença em SQL que indique qual o nome das estradas que compõem cada trajeto.
## Resolução

In [3]:
SELECT S.trajetoid, E.estradaid, E.nome
FROM Segmento S, Estrada E
WHERE S.estradaid = E.estradaid;

# Exercício 2
Escreva uma sentença em SQL que indique qual o código de todos os trajetos que passam por São Paulo.

## Resolução

In [4]:
SELECT S.trajetoid, E.estradaid, E.nome, E.cidade_origem, E.cidade_destino
FROM Segmento S, Estrada E
WHERE S.estradaid = E.estradaid AND (E.cidade_origem = 'SP' OR E.cidade_destino = 'SP');

In [5]:
SELECT DISTINCT S.trajetoid
FROM Segmento S, Estrada E
WHERE S.estradaid = E.estradaid AND (E.cidade_origem = 'SP' OR E.cidade_destino = 'SP');

# Exercício 3

Considerando que os trajetos não tem ciclos, escreva uma sentença SQL que mostre a última estrada de cada trajeto.

## Resolução

In [6]:
SELECT T.trajetoid, E.nome
FROM Trajeto T, Segmento S, Estrada E
WHERE T.trajetoid = S.trajetoid AND S.estradaid = E.estradaid AND T.cidade_destino = E.cidade_destino

# Exercício 4

Escreva uma sentença em SQL que calcule a quilometragem total de cada Trajeto.

## Resolução

### Passo 1 - Relacionando as três tabelas: Trajeto, Segmento e Estrada

In [7]:
SELECT T.trajetoid, S.ordem, E.nome, E.cidade_origem, E.cidade_destino, E.extensao_km
FROM Trajeto T, Segmento S, Estrada E
WHERE T.trajetoid = S.trajetoid AND
      S.estradaid = E.estradaid
ORDER BY T.trajetoid, S.ordem;

### Passo 2 - Agrupando e Somando

In [8]:
SELECT T.trajetoid, SUM(E.extensao_km)
FROM Trajeto T, Segmento S, Estrada E
WHERE T.trajetoid = S.trajetoid AND
      S.estradaid = E.estradaid
GROUP BY T.trajetoid;

# Exercício 5

Para que um trajeto seja consistente, a cidade de origem deste trajeto deve ser igual à cidade de origem cadastrada no primeiro segmento do respectivo trajeto. Escreva uma consulta SQL que mostre a identificação dos trajetos que não estão consistentes conforme este critério.

## Resolução

### Passo 1 - Primeiro segmento de cada trajeto

In [9]:
SELECT * FROM Segmento WHERE ordem=1;

### Passo 2 - Associando o primeiro segmento ao respectivo Trajeto e respectiva Estrada

In [10]:
SELECT T.trajetoid, S.ordem, E.nome, E.cidade_origem, E.cidade_destino, E.extensao_km
FROM Estrada E, Segmento S, Trajeto T
WHERE E.estradaid = S.estradaid AND
      S.trajetoid = T.trajetoid AND
      S.ordem = 1;

### Passo 3 - Apresentando aqueles que são diferentes

In [11]:
SELECT T.trajetoid
FROM Trajeto T, Segmento S, Estrada E
WHERE T.trajetoid = S.trajetoid AND
      S.estradaid = E.estradaid AND
      T.cidade_origem <> E.cidade_origem AND
      S.ordem = 1;

# Exercício 6

Outro critério para que um trajeto seja consistente é que a cidade de destino deste trajeto deve ser igual à cidade de destino cadastrada no último segmento do respectivo trajeto. Escreva uma consulta SQL que mostre a identificação dos trajetos que não estão consistentes conforme este critério.

## Resolução

### Passo 1 - Recuperando o número de ordem do último segmento de cada trajeto

In [12]:
SELECT trajetoid, MAX(ordem)
FROM Segmento
GROUP BY trajetoid;

A query anterior pode ser explorada pelo caminho da `View` ou pelo caminho do `SELECT Aninhado`.

### Caminho da `VIEW`
### Passo 2 - Transformando a Query em `VIEW`

In [13]:
CREATE VIEW Maior AS
SELECT trajetoid, MAX(ordem) ordem_maior
FROM Segmento
GROUP BY trajetoid;

SELECT * FROM Maior;

### Caminho da `VIEW`
### Passo 3 - Relacionando a `VIEW` com as demais tabelas

In [14]:
SELECT T.trajetoid
FROM Maior M, Trajeto T, Segmento S, Estrada E
WHERE T.trajetoid = S.trajetoid AND
      S.estradaid = E.estradaid AND
      M.trajetoid = T.trajetoid AND
      T.cidade_destino <> E.cidade_destino AND
      S.ordem = M.ordem_maior;

TJ003

### Caminho do `SELECT` Aninhado
### Passo 2 - Transformando a Query em `SELECT` aninhado e relacionando com as demais tabelas

In [15]:
SELECT T.trajetoid
FROM Trajeto T, Segmento S, Estrada E,
     (SELECT trajetoid, MAX(ordem) ordem_maior
      FROM Segmento
      GROUP BY trajetoid) M
WHERE T.trajetoid = S.trajetoid AND
      S.estradaid = E.estradaid AND
      M.trajetoid = T.trajetoid AND
      T.cidade_destino <> E.cidade_destino AND
      S.ordem = M.ordem_maior;

TJ003

# Exercício 7

Retorne os trajetos de menor quilometragem entre cada origem/destino diferente. Esta consulta deve apresentar para cada um dos trajetos selecionados: cidade origem, cidade destino, menor quilometragem entre elas. A quilometragem de cada trajeto é calculada pela soma da quilometragem de todas as estradas que compõem o trajeto.

## Resolução

### Passo 1 - Criando uma `VIEW` com as quilometragens totais

In [16]:
CREATE VIEW Distancia AS
SELECT T.trajetoid, T.cidade_origem, T.cidade_destino, SUM(E.extensao_km) total_km
FROM Trajeto T, Segmento S, Estrada E
WHERE T.trajetoid = S.trajetoid AND
      S.estradaid = E.estradaid
GROUP BY T.trajetoid;

SELECT * FROM Distancia;

### Passo 2 - Extraindo os mínimos da `VIEW`

Tal como na questão anterior, também seria possível o uso de SELECT Aninhado.

In [17]:
SELECT cidade_origem, cidade_destino, MIN(total_km)
FROM Distancia
GROUP BY cidade_origem, cidade_destino;

# Exercício 8

Modifique a tabela de Trajetos para acrescentar um campo extensao_km que conterá a quilometragem total do trajeto, escreva uma sentença que calcule e atualize o campo extensao_km da tabela de Trajetos, baseando-se na soma da quilometragem das estradas.

# Exercício 9

Retorne o nome das cidades que não aparecem na origem de nenhum segmento.